# Cooking Assistant

## openai
The library that will make it easier for us to connect directly to the openAI API.

## os and dotenv
The libraries that allow me to upload this directly to github without exposing my API credentials.

In [21]:
import csv
import os
import openai
from dotenv import load_dotenv

In [22]:
load_dotenv()

True

## Connecting to the openAI API

In [12]:
openai.api_key = os.getenv("OPEN_API_KEY")

## generate_recipe
This function formats the prompt that is sent to GPT3 (text-davinci-003). It takes variables that will eventually be available inputs on a webpage.

In [35]:
def generate_recipe(cooking_skill, place_city, preparation_time, time_of_day):
    prompt = f"Please provide a detailed {time_of_day} cooking recipe with ingredients and step-by-step directions, suitable for someone with {cooking_skill}, originating from {place_city}, and taking {preparation_time} time to prepare and cook. Please also keep it under 10 ingredients. \nPlease provide this in JSON format with the following variables:\nname, description, ingredients, directions"

    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        max_tokens=1000,
        n=1,
        temperature=0,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0
    )
    # return response.choices[0].text.strip()
    return response

In [75]:
cooking_skill = "Easy"
place_city = "Chicago"
preparation_time = "Short"
time_of_day = "Lunch"

## recipe_json
Here we will be executing the function by passing our 3 variables into the prompt, which is then sent to openAPI

In [76]:
recipe = generate_recipe(cooking_skill, place_city, preparation_time, time_of_day)

## JSON Structure
This JSON response will be used to fill in our webpage in an digestible format.

In [77]:
print(recipe)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n{\n    \"name\": \"Chicago-Style Italian Beef Sandwich\",\n    \"description\": \"This classic Chicago-style Italian beef sandwich is easy to make and takes only a short time to prepare and cook. It's a great lunch option for anyone looking for a quick and delicious meal.\",\n    \"ingredients\": [\n        \"1 lb. beef roast\",\n        \"1/2 cup beef broth\",\n        \"1/4 cup Italian dressing\",\n        \"1/4 cup giardiniera (pickled vegetables)\",\n        \"1/4 cup pepperoncini peppers\",\n        \"1/2 teaspoon garlic powder\",\n        \"1/2 teaspoon onion powder\",\n        \"1/2 teaspoon Italian seasoning\",\n        \"4 Italian rolls\",\n        \"Provolone cheese (optional)\"\n    ],\n    \"directions\": [\n        \"1. Preheat oven to 350\u00b0F.\",\n        \"2. Place beef roast in a baking dish and pour beef broth over it.\",\n        \"3. In a small bowl, 

In [79]:
strip_recipe = recipe.choices[0].text.strip()

In [80]:
import json

In [81]:
recipe_json = json.loads(strip_recipe)

In [82]:
filename = 'pastresponse.csv'

In [83]:
with open(filename, 'a', newline='') as past_responses:
    fieldnames = ['name', 'description', 'ingredients', 'directions']
    writer = csv.DictWriter(past_responses, fieldnames=fieldnames)
    writer.writerow({
        'name': recipe_json['name'],
        'description': recipe_json['description'],
        'ingredients': ", ".join(recipe_json['ingredients']),
        'directions': ", ".join(recipe_json['directions'])
    })